In [1]:
# KNN + Boarder Detection + Normal Direction Constrains

# 1) KNN search: several candidates of corresponding points
# 2) Boarder Detection + Normal Direction constrains => remove some bad correspondences

In [2]:
import math
import pickle
import os
import time
import scipy
import numpy as np
from scipy.optimize import minimize
from sklearn.neighbors import NearestNeighbors


## SMPL model
data_root   = '/home/yan/Data2/NOMO_Project_Summary/'
root_folder = data_root + 'MVA/MVA_rebuttal_code/Shape_Parameter_Optimization/'
import sys
sys.path.append(root_folder+'SMPL/SMPL_python_v.1.0.0/smpl/')
from smpl_webuser.serialization import load_model

smpl_model  = root_folder + 'SMPL/SMPL_python_v.1.0.0/smpl/models/basicModel_f_lbs_10_207_0_v1.0.0.pkl'
scan_folder =  data_root + 'Data/TC2_scan/ori_scan/re_name/dataset01/female/'
out_folder  = data_root + 'MVA/MVA_rebuttal_code/Shape_Parameter_Optimization/optimized_mesh/dataset01/'

# pre-process initial params
# param_folder = '/home/yan/Data2/3D_Body_Reconstruction/Dataset/scans/Optimized_Registered_NOMO3D_Dataset1/Original_A_Posed/parameter_male/'
param_folder = None
# load SMPL model
model = load_model(smpl_model)
scan_list = os.listdir(scan_folder)

deg2rad = 1.0 / 180 * math.pi
bnds = ((-20*deg2rad, -5*deg2rad), (-5*deg2rad,  5*deg2rad), (-90*deg2rad, -30*deg2rad), 
        (-20*deg2rad, -5*deg2rad), (-5*deg2rad,  5*deg2rad), (30*deg2rad,   90*deg2rad),
        (5*deg2rad,   30*deg2rad), (-30*deg2rad,-5*deg2rad), (0*deg2rad,    10*deg2rad),  
        (5*deg2rad,   30*deg2rad), (5*deg2rad,  30*deg2rad), (-10*deg2rad,   0*deg2rad),  
        (-5*deg2rad,   5*deg2rad), (-5*deg2rad,  5*deg2rad), (0*deg2rad,    15*deg2rad), 
        (-5*deg2rad,   5*deg2rad), (-5*deg2rad,  5*deg2rad), (-15*deg2rad,   0*deg2rad),
        (0*deg2rad,   10*deg2rad), (0*deg2rad,  10*deg2rad), (-5*deg2rad,    5*deg2rad),
        (-10*deg2rad,  0*deg2rad), (-10*deg2rad, 0*deg2rad), (-10*deg2rad,   0*deg2rad), 
        (-10*deg2rad,  0*deg2rad), (0*deg2rad,  10*deg2rad), (0*deg2rad,    10*deg2rad),
        (-3, 3), (-3, 3), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5))

In [3]:
def loadTarget(scanFile):
    ''' load NOMO3D scans, return Vertices and Faces'''
    with open(scanFile, 'r') as fp:
        content = fp.readlines()
    content = [x.strip() for x in content]

    targetV = []
    targetF = []
    for line in content:
        CC = line.split()
        if len(CC) > 0:
            if CC[0] == 'v':
                targetV.append([float(CC[1]), float(CC[2]), float(CC[3])])
            elif CC[0] == 'f':
                targetF.append([int(CC[1]), int(CC[2]), int(CC[3])])

    targetV = np.array(targetV)
    targetV = targetV - np.mean(targetV, 0)
    targetF = np.array(targetF, dtype=int) - 1 # .obj file, is 0-based, started from 0
    
    return targetV, targetF

def loadInitParam(paramFile):
    '''load initial parameters, including shape and pose from the previous experiments'''
    if paramFile == None:
        init_betas = np.zeros(10)
        init_pose = np.zeros(72)
        deg2rad = 1.0 / 180 * math.pi
        init_pose[48:60] = np.multiply(deg2rad, [-12, 2, -69, -12,-2, 69, 16, -18, 4, 16, 18, -4])
        init_pose[3:18] = np.multiply(deg2rad, [1, 0, 9, 1, 0, -8, 4, 4, 0, -3, -2, -2, -3, 2, 2])
    else:
        with open(paramFile, 'rb') as f:
            param_data = pickle.load(f)
        init_pose = param_data[:72]
        init_betas = param_data[72:]
    
    return init_pose, init_betas

def normalize_v3(arr):
    '''Normalize a numpy array of 3 component vectors shape=(n, 3)'''
    lens = np.sqrt(arr[:, 0]**2 + arr[:, 1]**2 + arr[:, 2]**2)
    arr[:, 0] /= lens
    arr[:, 1] /= lens
    arr[:, 2] /= lens
    return arr

def getNormals(V, F):
    ''' Creating a normal per triangle, is done by the cross product of two vectors on this triangle.
    Creating smooth normals, i.e. one normal per vertex is done 
    by averaging the normal of all the triangles one vertex is part of.
    
    https://sites.google.com/site/dlampetest/python/calculating-normals-of-a-triangle-mesh-using-numpy'''
    
    norm = np.zeros(V.shape, dtype=V.dtype)
    # Create an indexed view into the vertex array using the array of three indices for triangles
    tris = V[F] 
    # Calculate the normal for all the triangles, 
    # by taking the cross product of the vectors v1-v0, and v2-v0 in each triangle 
    n = np.cross( tris[::,1 ] - tris[::,0]  , tris[::,2 ] - tris[::,0] )
    n = normalize_v3(n)
    # Multiple triangles would then contribute to every vertex
    norm[F[:, 0]] += n
    norm[F[:, 1]] += n
    norm[F[:, 2]] += n
    norm = normalize_v3(norm)
    
    return norm

def getBoarderV(Faces):
    '''the boarder of the mesh are those edges which belong to only one polygon'''
    boundV = []
    edges01 = np.array([Faces[:, 0], Faces[:, 1]], dtype=int).T
    edges02 = np.array([Faces[:, 0], Faces[:, 2]], dtype=int).T
    edges03 = np.array([Faces[:, 1], Faces[:, 2]], dtype=int).T
    edge = np.sort(np.concatenate((edges01, edges02, edges03), axis=0))
    
    unique_e, n_counts = np.unique(edge, axis=0, return_counts=True)
    boarder_edge = unique_e[n_counts==1, :]
    boundV = np.unique(boarder_edge.flatten())
    
    return boundV

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'"""
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.absolute(np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)))

In [4]:
def meshDist(param):
    '''the distance between the SMPL model and the target scan'''
    
    model.betas[:] = param[27:]
    model.pose[48:60] = param[:12]
    model.pose[3:18] = param[12:27]
    
    modelF = model.f # 0-based, started from 0
    modelV = model.r - np.mean(model.r, 0)
    modelNorm = getNormals(modelV, modelF)
    
    # KNN correspondence search, find 5 nearest neighbour as the estimated correspondences
    N_neigh = 3
    _, t_idx = kdtree.query(modelV, k=N_neigh)
    
    # the correspondence should : 
    #      1) the similar normal, the angle is small
    #      2) the small distance,
    #      3) not at the boarder,
    
    W = np.ones((6890, 1), dtype=float) # 0 for bad correspondences, otherwise 1
    correspondV = np.zeros((6890, 3), dtype=float)
    
    dist = 0
    for idx in range(6890):
        m_normal = modelNorm[idx, :]
        good_neigh_idx = 0
        minAngle = 100
        for c_id in range(N_neigh):
            neigh_idx = t_idx[idx, c_id]
            t_normal = targetNorm[neigh_idx, :]
            angle = angle_between(t_normal, m_normal)
            if angle <= minAngle:
                minAngle = angle
                correspondV[idx, :] = targetV[neigh_idx, :]
                good_neigh_idx = neigh_idx
                
        dist_v = np.sqrt(np.sum(np.square(modelV[idx, :] - targetV[good_neigh_idx, :])))
        if dist_v > 0.1:           # 10 cm
            W[idx, :] = 0
        if minAngle > math.pi / 4: # 45 degree
            W[idx, :] = 0
        if good_neigh_idx in boarderV:
            W[idx, :] = 0
        
        dist = dist + dist_v * W[idx, :]
    # weighted mean distance between the model and the correspondences vertices
#     dist = np.sum(np.multiply(W, np.sqrt(np.sum(np.square(modelV - correspondV), axis=1)))) / np.count_nonzero(W)
    dist = dist / np.count_nonzero(W)
    
    return dist * 1000 # [mm]

In [5]:
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)


# start_time = time.time()

for scan in scan_list:
    scan_name = scan[:-4]

    targetV, targetF = loadTarget(scan_folder+scan_name+'.obj') # registered mesh, (6890, 3)
    boarderV = getBoarderV(targetF)                             # the vertices on the boarder edges
    targetNorm = getNormals(targetV, targetF)                   # the normal for every target vertices
    kdtree = scipy.spatial.cKDTree(targetV, leafsize=6890)      # KNN search
    
#     paramFile = param_folder + scan_name + '.pkl'               # None for female samples     
    paramFile = None
    init_pose, init_betas = loadInitParam(paramFile)            # female does not have the pre-param
    
    init_param        = np.zeros(37)                            # 27 part pose parameters + 10 shape param
    init_param[:12]   = init_pose[48:60]
    init_param[12:27] = init_pose[3:18]
    init_param[27:]   = init_betas
    
    start_time = time.time()
    res = minimize(meshDist, init_param, method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-6, 'disp': True})
    end_time = time.time()
    print "eclipse time [second] : ", end_time - start_time
    
    optimal_param = res.x
    print scan_name, " : ", optimal_param[27:]
    
    # save
    model.betas[:]    = optimal_param[27:]
    model.pose[48:60] = optimal_param[:12]
    model.pose[3:18]  = optimal_param[12:27]

    with open(out_folder + 'scan2Optimized_shape_pose/female/' + scan_name + '.obj', 'w') as fp:
        for v in model.r: # [m]
            fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )
        for f in model.f+1: # Faces are 1-based, not 0-based in obj files
            fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )

    optimal_param = np.zeros(82)
    optimal_param[72:]   = res.x[27:]
    optimal_param[48:60] = res.x[:12]
    optimal_param[3:18]  = res.x[12:27]

    with open(out_folder + 'scan2Optimized_shape_pose/female_param/' + scan_name + '.pkl', 'w') as fp:
        pickle.dump({'param_data': optimal_param}, fp)

# end_time = time.time()

# print "eclipse time [second] : ", end_time - start_time

eclipse time [second] :  10745.5192339
female_0135  :  [-0.15686288 -0.229764   -0.07308946  0.10054852  0.02711641 -0.15150211
 -0.03171508  0.01064662  0.01018151 -0.0034372 ]
eclipse time [second] :  10092.092834
female_0012  :  [ 2.99413682  1.31979279 -3.63932186  2.65284105  2.70093617 -2.07259052
  4.14449665  1.7214859  -0.53653552 -1.955996  ]
eclipse time [second] :  5585.69755197
female_0168  :  [-0.02399748  0.01940974 -0.02601477  0.02671743  0.01473291 -0.04425042
 -0.0280701   0.00318554  0.00687886  0.00170808]
eclipse time [second] :  4911.99483109
female_0164  :  [-0.45291476  0.84135448 -0.26682371 -0.03264948  0.10846709 -0.34850398
  0.04263741 -0.00937513  0.01581899  0.01173804]
eclipse time [second] :  15098.5551519
female_0094  :  [ 0.94776521  0.38889248 -1.13496903  0.7123047   0.22655811 -0.98759175
  0.49304723  0.66277345  0.46369898 -0.44129104]
eclipse time [second] :  16263.3587899
female_0087  :  [-2.6921138   2.77850762 -1.99563834  0.56539437  0.4350

eclipse time [second] :  7239.00663805
female_0114  :  [ 0.3018681  -0.21353803 -0.07847334  0.11578378  0.02593189 -0.21607994
 -0.0461521   0.07571868  0.03678955  0.01628188]
eclipse time [second] :  10607.4861522
female_0068  :  [ 0.37484865 -0.39530675  0.05464809  0.02815611  0.1019276  -0.20246463
  0.0634344   0.02141543  0.00184199  0.00098459]
eclipse time [second] :  5217.68793106
female_0152  :  [-0.34197101  0.74484764 -0.47075947  0.37121395  0.24119303 -1.07378244
 -0.53927985  0.17538797  0.15130403  0.07838302]
eclipse time [second] :  8039.09151196
female_0120  :  [ 1.16848535 -0.65097852  0.2084646   0.11819531 -0.12498597 -0.36040803
 -0.14728919  0.15765244  0.07696141 -0.07462366]
eclipse time [second] :  12091.8331511
female_0092  :  [ 0.86480304  0.79627959 -0.29647821  0.04719212 -0.2638418  -0.19706295
 -0.01534773  0.20253548  0.08539106 -0.10276772]
eclipse time [second] :  3427.82750988
female_0005  :  [-1.66443613e+00  1.26553570e-01 -2.36746128e-01  2.061

eclipse time [second] :  4966.90922308
female_0071  :  [ 0.11646506 -0.03963162 -0.03246297  0.015834    0.06858135 -0.21459788
 -0.02519268  0.00757723  0.00045888 -0.00647319]
eclipse time [second] :  4226.66273999
female_0080  :  [ 0.47975165  0.08874255 -0.05765336  0.05883327  0.08278219 -0.18384047
 -0.06408488  0.05081569  0.04303516  0.02522485]
eclipse time [second] :  5486.50051594
female_0053  :  [ 2.26311901  1.35826137 -0.88517691  0.44157817  0.5430301  -1.50534006
  0.32653293  0.41125909  0.11482283 -0.26101938]
eclipse time [second] :  3054.88164306
female_0006  :  [-0.0049726   0.06813905 -0.05760926 -0.01023685 -0.00206239 -0.20513211
  0.0781903   0.06454775 -0.00086891 -0.01569708]
eclipse time [second] :  8542.35742497
female_0175  :  [-0.0237948   0.09986593 -0.02633531 -0.00914336 -0.00654257  0.00196878
  0.00754695  0.00705897  0.0001947  -0.00380389]
eclipse time [second] :  7013.55693698
female_0013  :  [-0.45705025  0.99840471 -0.23863659 -0.09807684  0.056

eclipse time [second] :  11342.1406529
female_0088  :  [ 2.96043165e+00  2.05093258e+00 -1.23732119e+00  5.47779893e-01
  9.42581285e-01 -2.88546121e+00  1.06757985e-01  6.93422079e-01
  1.22894871e-03 -4.75820395e-01]
eclipse time [second] :  7124.52363706
female_0159  :  [ 2.64831468  2.13117349 -0.75235476  0.34379654  1.48428356 -4.36922731
 -1.69893901  0.6042549   0.11694511  0.15791999]
eclipse time [second] :  3087.35657406
female_0171  :  [ 1.02699082 -1.31765933  0.30152436  0.12689805  0.07414196 -0.30213
 -0.04171617  0.07047781  0.04347104 -0.02210664]
eclipse time [second] :  6165.09598088
female_0176  :  [ 3.          0.75730626  0.11570851  1.05520806 -0.0304478  -1.92840905
 -0.90182296  0.64657001  0.24570252  0.03477691]
eclipse time [second] :  5465.64333892
female_0073  :  [-0.52161604  0.38903849 -0.16605496 -0.02266431  0.06644772 -0.67485213
 -0.03714591 -0.01659454  0.01351283 -0.02522064]
eclipse time [second] :  2432.17126203
female_0167  :  [ 0.01176274 -0.1

In [6]:
print "done"

done
